# 🚀 ML Model Deployment

**Project**: Model Deployment and Production ML  
**Level**: Expert  
**Frameworks**: Flask, Streamlit, FastAPI, Docker  

## 📋 Project Overview

This project demonstrates how to deploy machine learning models to production using various frameworks and deployment strategies. We'll learn:

- Building REST APIs with Flask and FastAPI
- Creating interactive dashboards with Streamlit
- Containerization with Docker
- Cloud deployment strategies
- MLOps best practices

Let's deploy our models to production! 🌐

## 1. Import Libraries and Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Model persistence
import joblib
import pickle

# Web frameworks (for demonstration)
import requests
import json

# Utilities
import os
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🚀 Ready for model deployment!")

## 2. Model Training and Preparation

In [ ]:
# Load and prepare data
print("📊 Loading Iris dataset...")
iris = load_iris()
X, y = iris.data, iris.target

# Create DataFrame for analysis
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df['species'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print(f"Dataset shape: {df.shape}")
print(f"Features: {list(iris.feature_names)}")
print(f"Classes: {list(iris.target_names)}")

# Display first few rows
df.head()

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Train model
print("\n🤖 Training Random Forest model...")
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=5
)

model.fit(X_train, y_train)

# Evaluate model
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)
y_pred = model.predict(X_test)

print(f"✅ Model trained successfully!")
print(f"Training accuracy: {train_accuracy:.3f}")
print(f"Test accuracy: {test_accuracy:.3f}")

# Detailed classification report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
bars = plt.bar(feature_importance['feature'], feature_importance['importance'])
plt.title('🌟 Feature Importance in Random Forest Model', fontweight='bold', fontsize=14)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("🔍 Feature Importance Ranking:")
for _, row in feature_importance.iterrows():
    print(f"• {row['feature']}: {row['importance']:.3f}")

## 3. Model Serialization and Persistence

In [ ]:
# Create models directory
os.makedirs('models', exist_ok=True)

print("💾 Saving model and metadata...")

# Save model using joblib (recommended for scikit-learn)
joblib.dump(model, 'models/iris_model.pkl')
print("✅ Model saved as 'iris_model.pkl'")

# Save feature names
with open('models/feature_names.pkl', 'wb') as f:
    pickle.dump(iris.feature_names, f)
print("✅ Feature names saved")

# Save target names
with open('models/target_names.pkl', 'wb') as f:
    pickle.dump(iris.target_names, f)
print("✅ Target names saved")

# Save model metadata
metadata = {
    'model_type': 'RandomForestClassifier',
    'training_date': datetime.now().isoformat(),
    'train_accuracy': float(train_accuracy),
    'test_accuracy': float(test_accuracy),
    'n_features': len(iris.feature_names),
    'n_classes': len(iris.target_names),
    'feature_names': list(iris.feature_names),
    'target_names': list(iris.target_names),
    'model_params': model.get_params()
}

with open('models/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ Model metadata saved")

# Check file sizes
model_size = os.path.getsize('models/iris_model.pkl') / 1024  # KB
print(f"\n📊 Model file size: {model_size:.1f} KB")

In [ ]:
# Test model loading
print("🔄 Testing model loading...")

# Load model
loaded_model = joblib.load('models/iris_model.pkl')

# Load metadata
with open('models/metadata.json', 'r') as f:
    loaded_metadata = json.load(f)

# Test prediction
test_sample = X_test[0:1]  # First test sample
original_pred = model.predict(test_sample)[0]
loaded_pred = loaded_model.predict(test_sample)[0]

print(f"✅ Model loaded successfully!")
print(f"Original prediction: {iris.target_names[original_pred]}")
print(f"Loaded model prediction: {iris.target_names[loaded_pred]}")
print(f"Predictions match: {original_pred == loaded_pred}")

print(f"\n📋 Loaded Metadata:")
for key, value in loaded_metadata.items():
    if key != 'model_params':  # Skip detailed params for readability
        print(f"• {key}: {value}")

## 4. API Testing and Validation

In [ ]:
# Create sample prediction function (simulating API)
def predict_species(features, model=loaded_model, target_names=iris.target_names):
    """
    Simulate API prediction function
    """
    try:
        # Validate input
        if len(features) != 4:
            return {'error': f'Expected 4 features, got {len(features)}'}
        
        # Convert to numpy array
        features_array = np.array(features).reshape(1, -1)
        
        # Make prediction
        prediction = model.predict(features_array)[0]
        prediction_proba = model.predict_proba(features_array)[0]
        
        # Prepare response
        response = {
            'prediction': int(prediction),
            'prediction_label': target_names[prediction],
            'probabilities': {
                target_names[i]: float(prob) 
                for i, prob in enumerate(prediction_proba)
            },
            'confidence': float(max(prediction_proba)),
            'timestamp': datetime.now().isoformat()
        }
        
        return response
    
    except Exception as e:
        return {'error': str(e)}

# Test with different examples
test_cases = [
    {
        'name': 'Typical Setosa',
        'features': [5.1, 3.5, 1.4, 0.2],
        'expected': 'setosa'
    },
    {
        'name': 'Typical Versicolor',
        'features': [5.7, 2.8, 4.1, 1.3],
        'expected': 'versicolor'
    },
    {
        'name': 'Typical Virginica',
        'features': [6.2, 2.8, 4.8, 1.8],
        'expected': 'virginica'
    }
]

print("🧪 Testing API prediction function...\n")

for test_case in test_cases:
    result = predict_species(test_case['features'])
    
    print(f"📝 Test: {test_case['name']}")
    print(f"   Input: {test_case['features']}")
    
    if 'error' in result:
        print(f"   ❌ Error: {result['error']}")
    else:
        print(f"   🎯 Prediction: {result['prediction_label']}")
        print(f"   📊 Confidence: {result['confidence']:.1%}")
        print(f"   ✅ Correct: {result['prediction_label'] == test_case['expected']}")
    
    print()

## 5. Deployment Strategies Overview

In [ ]:
# Display deployment options
deployment_options = {
    'Flask REST API': {
        'description': 'Simple web API for model predictions',
        'use_case': 'Backend service for applications',
        'pros': ['Simple', 'Lightweight', 'Flexible'],
        'cons': ['Manual scaling', 'Basic features']
    },
    'Streamlit Dashboard': {
        'description': 'Interactive web application',
        'use_case': 'Business dashboards and demos',
        'pros': ['No web dev needed', 'Interactive', 'Fast prototyping'],
        'cons': ['Limited customization', 'Not for APIs']
    },
    'FastAPI Service': {
        'description': 'High-performance async API',
        'use_case': 'Production microservices',
        'pros': ['Fast', 'Auto docs', 'Type validation'],
        'cons': ['More complex', 'Learning curve']
    },
    'Docker Container': {
        'description': 'Containerized deployment',
        'use_case': 'Cloud deployment and scaling',
        'pros': ['Portable', 'Scalable', 'Consistent'],
        'cons': ['Container overhead', 'Complexity']
    }
}

print("🚀 Deployment Options Overview:\n")

for option, details in deployment_options.items():
    print(f"📦 {option}")
    print(f"   Description: {details['description']}")
    print(f"   Use Case: {details['use_case']}")
    print(f"   Pros: {', '.join(details['pros'])}")
    print(f"   Cons: {', '.join(details['cons'])}")
    print()

## 6. Production Considerations

In [ ]:
# Production checklist
production_checklist = {
    '🔒 Security': [
        'API authentication and authorization',
        'Input validation and sanitization',
        'HTTPS encryption',
        'Rate limiting and DDoS protection'
    ],
    '📊 Monitoring': [
        'Request/response logging',
        'Performance metrics (latency, throughput)',
        'Model drift detection',
        'Error tracking and alerting'
    ],
    '⚡ Performance': [
        'Response time optimization',
        'Caching strategies',
        'Load balancing',
        'Auto-scaling configuration'
    ],
    '🔄 Reliability': [
        'Health checks and heartbeats',
        'Graceful error handling',
        'Circuit breakers',
        'Backup and recovery plans'
    ],
    '🧪 Testing': [
        'Unit tests for prediction logic',
        'Integration tests for API endpoints',
        'Load testing for performance',
        'A/B testing for model versions'
    ]
}

print("✅ Production Deployment Checklist:\n")

for category, items in production_checklist.items():
    print(f"{category}")
    for item in items:
        print(f"   • {item}")
    print()

## 7. Next Steps and Resources

In [ ]:
# Display next steps
next_steps = {
    '🚀 Immediate Actions': [
        'Run the Flask app: python app.py',
        'Launch Streamlit: streamlit run streamlit_app.py',
        'Build Docker image: docker build -t ml-api .',
        'Test API endpoints with curl or Postman'
    ],
    '☁️ Cloud Deployment': [
        'Deploy to Heroku for quick hosting',
        'Use AWS ECS/Fargate for containers',
        'Try Google Cloud Run for serverless',
        'Explore Azure Container Instances'
    ],
    '🔧 Advanced Features': [
        'Add model versioning with MLflow',
        'Implement A/B testing framework',
        'Set up CI/CD pipelines',
        'Add real-time monitoring dashboards'
    ],
    '📚 Learning Resources': [
        'MLOps: Machine Learning Operations',
        'Kubernetes for ML workloads',
        'Model serving with TensorFlow Serving',
        'Monitoring with Prometheus and Grafana'
    ]
}

print("🎯 Next Steps and Recommendations:\n")

for category, items in next_steps.items():
    print(f"{category}")
    for item in items:
        print(f"   • {item}")
    print()

print("🎉 Congratulations! You've completed the ML Model Deployment project!")
print("🚀 Your model is now ready for production deployment!")